In [ ]:
import torch
from PIL import Image
import glob
import os

from modules.car_model import CarModel, ImageMask
from modules.renderer import Renderer
from modules.img_utils import tensor2img, img2tensor, interpolate, sum_images, union_img

# device_id = 'cpu'
device_id = 6
device = torch.device(device_id)
print(device)

In [ ]:
## 汽车模型
obj_file = '../models/rangerover/rangerover.obj'

car = CarModel(device, obj_file)

img_size = 640

## 渲染器
renderer_config = {
    'img_size': img_size,
    "rander_scale": 0.345,
    "rander_car_at": (0, 0, 0),
    'fov': 60,
}

render = Renderer(device, **renderer_config)
detector_type = 'yolov9'
detector_config = {
    'weight': '../checkpoints/yolov9-c-converted.pt',
}

from modules.detector import Detector
detector = Detector(device, img_size, detector_type, **detector_config)

In [ ]:
## 数据生成器
data_config = {
    # 距离
    'Distance': (6, 20),
    # 俯仰角 0~90 deg
    'Sigma': (0, 45),
    # 方位角
    #  +135    Y +90    +45
    #       /----R---+    
    # +180 (F [car] B|  +0 -> X
    #       \----L---+
    #  -135     -90     -45
    'Theta': (-180, +180),
    # 'Theta': (45, 360-45),
    # 'Theta': (-190,  -30),
    # 
    'Roll': (-5, +5),
}

target_class = [2, 5, 7]

## 检测参数
tester_config = {
    'bound_size': 0.2,
    'confidence_threshold': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    'sample_cnt': 0,
}

val_size = 1000

from modules.dataset_generator import train_data_sample, SimpleBackgroundDataset
from torch.utils.data import DataLoader
from modules.attack_tests import AttackTester
from modules.logger import NullLogger

logger=NullLogger()

def test_asr_car(background_dir='white.png', tex_img=None, mask_img='mask0'):
    car_tex_mask = ImageMask(device, f"../models/rangerover/{mask_img}.png",
    random_brightness= (0.6, 1.2), 
    random_bias_r= 0.1,
)

    val_data_set = train_data_sample(device, val_size, f'../data/{background_dir}', img_size, **data_config)
    test_dataset = SimpleBackgroundDataset(device, val_data_set, car_tex_mask, car, render)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

    texture_tester = AttackTester(test_dataloader, detector, logger, 
                                target_class=target_class, img_size=img_size, 
                                **tester_config)
    
    if tex_img is None:
        tex = None
    else:
        if isinstance(tex_img, str): 
            tex = img2tensor(Image.open(tex_img), device)
        else:
            tex = tex_img

    test_dataset.set_texture(tex)
    return texture_tester.test('')

In [ ]:
## 渲染

def get_bg(background_dir, bg_conf):
    bg_img = img2tensor(Image.open(f'../data/{background_dir}'), device)
    xx = bg_img.shape[1]
    yy = bg_img.shape[0]
    # if bg_conf is None:
    #     x1, x2, y1, y2 = 0, xx, 0, yy
    # else:
    x, y, size = bg_conf
    w=min(xx,yy)
    x1, x2, y1, y2 = int(xx-x*xx-size*w), int(xx-x*xx), int(yy-y*yy-size*w), int(yy-y*yy)
    # x1, x2, y1, y2 = int(x*xx), int(x*xx+size*w), int(y*yy), int(y*yy+size*w)
    if x2 > xx or y2 > yy:
        print(xx, yy, x1, x2, y1, y2)
        raise
    bg = interpolate(bg_img[y1:y2, x1:x2, :], [img_size, img_size])
    return bg

def render_car_only(tex_img, mask_img, car_pos):
    car_tex_mask = ImageMask(device, f"../models/rangerover/{mask_img}.png")

    if tex_img is None:
        tex = None
    else:
        if isinstance(tex_img, str): 
            tex = img2tensor(Image.open(tex_img), device)
        else:
            tex = tex_img
        tex = car_tex_mask.mask(interpolate(tex, car.uv_shape), pos=car_pos)
        
        
    mesh = car.to_mesh(tex)
    imgs = render.render(mesh, car_pos).squeeze(0)

    return imgs

def mix_bg(imgs, background_dir, bg_conf):
    bg = get_bg(background_dir, bg_conf)

    rgb = imgs[..., :3]                                 # [N, N, 3] 
    alpha = imgs[..., 3:]                               # [N, N, 1]
    img = (rgb * alpha + bg * (1 - alpha)).permute((2, 0, 1))   # [3, N, N]

    return img

def render_car(background_dir='white.png', bg_conf=(0,0,1), 
               tex_img=None, mask_img='mask0',
                # (distance, sigma, theta, roll)
               car_pos=(7, 35, -120, 0), show=True):
    img = mix_bg(render_car_only(tex_img, mask_img, car_pos), background_dir, bg_conf)

    if show:
        tensor2img(img).show()
    return img


def get_by_name(name):
    res =glob.glob(f'../logs/*{name}*')
    if len(res)!=1:
        print(name)
        print(res)
        raise
    return res[0] + '/Epoch_20.png'

def get_by_name2(name):
    res =glob.glob(f'../logs/*{name}')
    if len(res)!=1:
        print(name)
        print(res)
        raise
    return res[0] + '/Epoch_20.png'

# img=render_car()




In [ ]:

# graph 2cd
# img=render_car(
#     background_dir = 'new/city2.jpg',
#     bg_conf = (0.51, 0.4, 0.6),
#     # tex_img = get_by_name('full_city0'),
#     tex_img = get_by_name('full_city2'),
#     mask_img = 'mask_eext',
#     car_pos = (7, 20, -160, +0),
# )

img=render_car(
    background_dir = 'new/forest3.jpg',
    bg_conf = (0.2, 0.3, 0.7),
    tex_img = get_by_name('full_forest3'),
    mask_img = 'mask_eext',
    car_pos = (7, 20, -130, +0),
)

# img=render_car(
#     background_dir = 'new/forest4.jpg',
#     bg_conf = (0.3, 0.5, 0.5),
#     tex_img = get_by_name('full_forest4'),
#     mask_img = 'mask_eext',
#     car_pos = (8, 20, -160, +0),
# )

# raise

In [ ]:

# # graph 2e
# img=render_car(
#     background_dir = 'new/desert0.jpg',
#     bg_conf = (0.0, 0.0, 1),
#     # tex_img = glob.glob(f'../logs/*nosm_desert0*')[0] + '/Epoch_20.png',
#     tex_img = glob.glob(f'../logs/*full_desert0*')[0] + '/Epoch_20.png',
#     # tex_img = glob.glob(f'../logs/241208_22.31.16_v4')[0] + '/Epoch_0.png',
#     mask_img = 'mask_eext',
#     car_pos = (8, 30, -120, +0),
# )

# # graph 5a
# # 干净，随机，FCA，DTA，随机迷彩，半身，全身
# conf = [
#     # ('city2', (0.52, 0.5, 0.5), (7, 20, -150, +0)),
#     # ('snow0', (0.0, 0.4, 0.6), (7, 15, 40, +0)),
#     ('beach0', (0.0, 0.4, 0.6), (8, 35, 100, +0)),
#     ('desert0', (0.2, 0.2, 0.8), (7, 30, +120, +0)),
#     ('forest0', (0.18, 0.55, 0.5), (6.5, 20, -145, +0)),
#     ('night0', (0.0, 0.4, 0.7), (7.5, 20, -20, +0)),

#     ('snow0', (0.2, 0.36, 0.6), (7, 15, 40, +0)),
#     ('city2', (0.1, 0.5, 0.7), (7, 20, -130, +0)),
# ]

# texes = [
#     'Clean', 
#     'Rand2', 'RandSM2_', 
#     'FCA', 'DTA', 
#     'NoSM', 
#     'Full_', 
#     # 'Half_', 
# ]


# asrs = {}
# for s, bg_conf, car_pos in conf:
#     rres = []
#     for tname in texes:

#         name = f'{tname}_{s}'.replace('__', '_')
#         background_dir = f'new2/{s}.jpg'
#         tex_img = f'../logs/_t/{(tname + s if tname.endswith("_") else tname)}.png'

#         img=render_car(
#             mask_img = 'mask_eext',
#             show=False,

#             background_dir = background_dir,
#             bg_conf=bg_conf,
#             car_pos=car_pos,
            
#             tex_img = tex_img,
#         )
#         # tensor2img(img).save(f'../logs/_r/{name}.png')

#         res = detector.run(img.unsqueeze(0))
#         # print(res)
#         img=detector.show(img, res, 0.5, target_class, 8, 50)

#         rres.append(img)
#         img.save(f'../logs/_r/{name}_yolov9_conf0.5.png')

#         asr = test_asr_car(background_dir, tex_img, 'mask_eext')
#         asrs[name] = asr

#     sum_images(rres, 400).show()

# import pickle
# with open(f'./ASR2.pkl', 'wb') as f:
#     pickle.dump(asrs, f)

In [ ]:

import pickle
import shutil
shutil.rmtree('../logs/_pl')
os.mkdir('../logs/_pl')

from modules.texture_optim import GumbleSoftmaxTexture

# places 
# 干净，随机，FCA，DTA，随机迷彩，半身，全身
conf = [
    ('lake', (0.0, 0.4, 0.6), (8, 35, 100, +0)),
    # ('canyon', (0.2, 0.2, 0.8), (7, 30, +120, +0)),
    # ('alley', (0.2, 0.2, 0.8), (7, 30, +120, +0)),
    # ('swamp', (0.18, 0.55, 0.5), (6.5, 20, -145, +0)),
    # ('field', (0.0, 0.4, 0.7), (7.5, 20, -20, +0)),
    # ('highway', (0.2, 0.36, 0.6), (7, 15, 40, +0)),
    # ('village', (0.1, 0.5, 0.7), (7, 20, -130, +0)),
]

texes = [
    # 'Clean', 
    # 'Rand2', 
    # 'FCA', 'DTA', 
    # 'NoSM', 
    'Full_', 
    'RandSM2_', 
]

asrs = {}
for sclass, bg_conf, car_pos in conf:
    for s in os.listdir(f'../data/places_s/{sclass}'):
        rres = []

        background_dir = f'../data/places_clean/{s}'

        raw_name = s.split('.')[0]
        dir = glob.glob(f'../logs/*{raw_name}')[0]

        raw_name = raw_name[2:] + '_' + raw_name[:2]
        with open(f'{dir}/info.pkl','rb') as f:
            obj = pickle.load(f)
            main_colors=obj['main_colors']
            main_color_cnt=obj['main_color_cnt']
            tau=obj['tau']

        shutil.copy(f'{dir}/main_colors.png', f'../logs/_pl/_mc_{raw_name}.png')
        
        texture = GumbleSoftmaxTexture(device, [512, 512], main_color_cnt, tau)
        texture.set_main_colors(main_colors)


        for tname in texes:
            name = f'{raw_name}_{tname}'.replace('__', '_')
            if tname == 'RandSM2_':
                tex_img = texture.texture()
            elif tname == 'Full_':
                tex_img =  f'{dir}/Epoch_20.png'
            else:
                tex_img = f'../logs/_t/{tname}.png'

            # img=render_car(
            #     mask_img = 'mask_eext',
            #     show=False,

            #     background_dir = background_dir,
            #     # bg_conf=bg_conf,
            #     car_pos=car_pos,
                
            #     tex_img = tex_img,
            # )
            # # tensor2img(img).save(f'../logs/_r/{name}.png')

            # res = detector.run(img.unsqueeze(0))
            # # print(res)
            # img=detector.show(img, res, 0.5, target_class, 8, 50)

            # rres.append(img)
            # img.save(f'../logs/_pl/{name}_yolov9_conf0.5.png')

            asr = test_asr_car(background_dir, tex_img, 'mask_eext')
            asrs[name] = asr

        # sum_images(rres, 400).show()

import pickle
with open(f'../logs/_ASR_pl.pkl', 'wb') as f:
    pickle.dump(asrs, f)

raise

In [ ]:

# CAM
# 干净，随机，FCA，DTA，随机迷彩，半身，全身
conf = [
    # ('city2', (0.52, 0.5, 0.5), (7, 20, -150, +0)),
    # ('snow0', (0.0, 0.4, 0.6), (7, 15, 40, +0)),
    ('beach0', (0.0, 0.4, 0.6), (8, 35, 100, +0)),
    ('desert0', (0.2, 0.2, 0.8), (7, 30, +120, +0)),
    ('forest0', (0.18, 0.55, 0.5), (6.5, 20, -145, +0)),
    ('night0', (0.0, 0.4, 0.7), (7.5, 20, -20, +0)),

    ('snow0', (0.2, 0.36, 0.6), (7, 15, 40, +0)),
    ('city2', (0.1, 0.5, 0.7), (7, 20, -130, +0)),
]

texes = [
    'Clean', 
    'Rand2', 'RandSM2_', 
    'FCA', 'DTA', 
    'NoSM', 
    'Full_', 
    # 'Half_', 
]

for sigma in [30]:
    for theta in range(36):
# for sigma in [0, 15]:
#     for theta in range(2):
        theta = theta * 10

        print(sigma, theta)
        os.makedirs(f'../logs/_cam/{sigma}/{theta}/', exist_ok=True)
                
        for tname in texes:
            for s, bg_conf, _ in conf:
                name = f'{tname}_{s}'.replace('__', '_')
                background_dir = f'new2/{s}.jpg'
                tex_img = f'../logs/_t/{(tname + s if tname.endswith("_") else tname)}.png'

                img=render_car(
                    mask_img = 'mask_eext',
                    show=False,

                    background_dir = background_dir,
                    bg_conf=bg_conf,
                    car_pos=(7, sigma, theta, 0),
                    
                    tex_img = tex_img,
                )

                name = '_'.join(name.split('_')[::-1])
                tensor2img(img).save(f'../logs/_cam/{sigma}/{theta}/{name}_{sigma}_{theta}.png')
    

In [ ]:
raise

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def create_image_grid(images, spacing):
    # 图片数量
    num_images = len(images)
    
    # 每行最大图片数和行数计算
    row_images = 2
    col_images = (num_images + 1) // row_images  # 列数，向上取整
    image_width, image_height = images[0].size
    
    # 创建新图像的尺寸
    grid_width = col_images * image_width + (col_images - 1) * spacing
    grid_height = row_images * image_height + row_images * spacing + 30 * row_images  # 每个图片下方留出空间

    # 创建新图像
    grid_image = Image.new('RGB', (grid_width, grid_height), (255, 255, 255))
    draw = ImageDraw.Draw(grid_image)
    
    # 加载字体（可以使用默认字体，也可以指定其他字体）
    font = ImageFont.truetype("./modules/consola.ttf", size=40)
    
    # 将图片添加到新图像中
    for index, img in enumerate(images):
        # 计算当前图片的位置
        row = index // col_images
        col = index % col_images
        
        x_position = col * (image_width + spacing)
        y_position = row * (image_height + spacing) + row * 30  # 图片下方留出空间
        
        # Paste the image
        grid_image.paste(img, (x_position, y_position))
        
        # 绘制标号
        label = f"({chr(97 + index)})"  # chr(97) = 'a'
        
        # 获取文本边界框
        text_bbox = draw.textbbox((0, 0), label, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        
        # 计算文本位置，使其水平居中
        text_x = x_position + (image_width - text_width) / 2
        text_y = y_position + image_height  # 在图片下方绘制
        draw.text((text_x, text_y), label, fill="black", font=font)
    
    return grid_image

# images = [Image.new('RGB', (640, 640), (i * 20, i * 10, i * 5)) for i in range(6)]  # 生成6个示例图片
    
# # 创建排列图像
# result_image = create_image_grid(images, spacing=50)

# # 保存结果图像
# result_image.save('result_image.png')
# result_image.show()  # 显示结果图像

In [ ]:
raise

In [ ]:
# ns

# 干净，随机，FCA，DTA，随机迷彩，半身，全身，参数
conf = [
    ('beach0', (0.0, 0.4, 0.6), (8, 35, 100, +0)),
    ('desert0', (0.2, 0.2, 0.8), (7, 30, +120, +0)),
    ('forest0', (0.18, 0.55, 0.5), (6.5, 20, -145, +0)),
    ('night0', (0.0, 0.4, 0.7), (7.5, 20, -20, +0)),

    ('snow0', (0.2, 0.36, 0.6), (7, 15, 40, +0)),
    ('city2', (0.1, 0.5, 0.7), (7, 20, -130, +0)),
]

texess = {
    'Graph_3c': [
        'Clean', 
        'Rand2', 'RandSM2_', 
        'FCA', 'DTA', 
        'NoSM', 'Full_', 
        # 'Half_', 
    ],
    'MCCnt': ['MCCnt@4_', 'MCCnt@6_', 'MCCnt@8_', 'MCCnt@10_', 'MCCnt@12_', 'MCCnt@14_', 'MCCnt@16_', 'MCCnt@20_', 'MCCnt@24_'],
    'Tau': ['Tau@0.1_', 'Tau@0.15_', 'Tau@0.2_', 'Tau@0.3_', 'Tau@0.4_', 'Tau@0.5_', 'Tau@0.6_', 'Tau@0.8_', 'Tau@1.0_'],
    'Ltv': ['Ltv@0_', 'Ltv@1_', 'Ltv@2_', 'Ltv@5_', 'Ltv@10_', 'Ltv@20_', 'Ltv@50_', 'Ltv@100_', 'Ltv@200_'],
}


poss = [
    # (7, 18, 45, 0),
    # (7, 18, 135, 0),
    (7, 18, 225, 0),
    (7, 18, 315, 0),
    (7, 35, 0, 0),
    # (7, 35, 90, 0),
    (7, 35, 180, 0),
    (7, 35, 270, 0),
]

for name in texess:
    texes = texess[name]
    dir=f'../logs/_ns/{name}'
    os.makedirs(dir, exist_ok=True)
    for s, bg_conf, best_pos in conf:
        background_dir = f'new/{s}.jpg'

        for i, car_pos in enumerate([best_pos, *poss]):
            rres = [tensor2img(get_bg(background_dir, bg_conf))]

            for tname in texes:
                tex_img = f'../logs/_t/{(tname + s if tname.endswith("_") else tname)}.png'
                img=render_car(
                    mask_img = 'mask_eext',
                    show=False,

                    background_dir = background_dir,
                    bg_conf=bg_conf,
                    car_pos=car_pos,
                    
                    tex_img = tex_img,
                )

                rres.append(tensor2img(img))

            img=create_image_grid(rres, 50)
            
            # name = f'{tname}_{s}'.replace('__', '_')
            img.save(f'{dir}/{s}_{i}.png')
            
            # img.show()
            # raise


In [ ]:
raise

In [ ]:

import shutil

keys={
    'MCCnt': ('main_color_cnt', [4,6,8,10, 12 ,14,16,20,24]),
    'Tau': ('tau', [0.1,0.15,0.2,0.3, 0.4 ,0.5,0.6,0.8,1.0]),
    'Ltv': ('n_smooth', [0,1,2,5, 10 ,20,50,100,200]),
}
for name, (key, vals) in keys.items():
    fl=[]
    for val in vals:
        fl.append(f'{name}@{val}_')
        for background in ['city2.jpg']:
        # for background in ['forest0.jpg','desert0.jpg','snow0.jpg','city0.jpg','night0.jpg','beach0.jpg']:
            vv=background.split('.')[0]
            shutil.copy(get_by_name2(f'v_{key}_{val}'), 
                        f'../logs/_t/{name}@{val}_{vv}.png')
    print(fl)


# for s, bg_conf, car_pos in conf:
#     shutil.copy(get_by_name(f'phy_{s}'), f'../logs/_t/Phy_{s}.png') 

In [ ]:
raise

res = detector.run(img.unsqueeze(0))
print(res)
detector.show(img, res, conf=0.2)